In [1]:
baseline = [
    './log.20190801-154135/',
    './log.archive/log.20190727-191215/',
    './log.20190801-024113/'
]

In [2]:
import os
import pickle
from pprint import pprint
from emr.utils import Namespace


global tmp_emr
def show_setup(log_path):
    with open(os.path.join(log_path, 'config.pkl'), 'rb') as fin:
        config = Namespace(pickle.load(fin))
    ret_config = config.deepclone()

    source = config.pop('Source')
    display(Markdown('## Setup'))
    pprint(config)
    if 'emr' in source:
        global tmp_emr
        tmp_emr = source.emr
        %pycat tmp_emr

    return ret_config

In [3]:
import json
from glob import glob
import re
from collections import defaultdict
import pandas as pd
from IPython.display import display, HTML, Markdown
from emr.conll_stat import Conll04Stat


def show_result(log_path, config=None):
    display(Markdown('## Result'))

    metrics_path = sorted(glob(os.path.join(log_path, 'metrics_epoch_*.json')), key=lambda p: os.path.getmtime(p)).pop()
    with open(metrics_path, 'r') as fin:
        metrics = json.load(fin)
        #pprint(metrics)

    def pop_regex(container, regex):
        if not isinstance(regex, re.Pattern):
            regex = re.compile(regex, re.MULTILINE | re.DOTALL)
        for key in list(container):
            match = regex.fullmatch(key)
            if match:
                yield match, container.pop(key)

    preserve_string = {'live_in': 'live-in',
                       'located_in': 'located-in',
                       'orgbase_on': 'orgbase-on',
                       'work_for': 'work-for'}

    def preserve(string):
        for k, v in preserve_string.items():
            string = string.replace(k, v)
        return string

    metrics_suffix = [
        'F1',
        'P',
        'R',
        'Accuracy',
        'loss',
    ]

    global_names = [
        'training_cpu_memory_MB',
        'training_duration',
        'training_epochs',
        'training_gpu_0_memory_MB',
        'training_gpu_1_memory_MB',
        'training_start_epoch',
        'best_epoch',
        'epoch',
        'peak_cpu_memory_MB',
        'peak_gpu_0_memory_MB',
        'peak_gpu_1_memory_MB',
    ]

    fields = [
        ('best', {'best'}),
        ('train', {'training', 'validation'}),
        ('inference', {'i'}),
    ]

    entities = {'people':'Peop', 'organization':'Org', 'location':'Loc', 'other':'Other'}
    relations = {'kill':'Kill', 'live-in':'Live_In', 'work-for':'Work_For', 'orgbase-on':'OrgBased_In', 'located-in':'Located_In'}
    er = entities.copy()
    er.update(relations)
    def parse_fields(string_list):
        field_list = ['',] * 4
        for i, (field, field_set) in enumerate(fields):
            if field:
                for j, string in enumerate(list(string_list)):
                    if string in field_set:
                        field_list[i] = string
                        string_list.pop(j)
        field_list[-1] = string_list.pop().strip()
        assert not string_list
        return field_list
    entity_table = pd.DataFrame()
    best_epoch = metrics['best_epoch']

    global_data = {}
    for name in global_names:
        for match, value in pop_regex(metrics, name):
            global_data[match.string] = value
    global_table = pd.DataFrame.from_dict(global_data, orient='index')
    display(Markdown('### Global information'))
    display(HTML(global_table.to_html(header=False,)))

    if config:
        test_path = os.path.join(config.Data.relative_path, config.Data.valid_path)
        stat = Conll04Stat()
        stat_result = dict(stat(test_path))
        
        #pprint(stat_result)
    else:
        stat_result = None

    P = {}
    TP = {}
    FP = {}
    TPi = {}
    FPi = {}

    for suffix in metrics_suffix:
        display(Markdown('#### ' + suffix))
        entity_dict = defaultdict(list)
        relation_dict = defaultdict(list)
        other_dict = defaultdict(list)
        entity_index = []
        relation_index = []

        total_dict = {}
        for match, value in pop_regex(metrics, r'(.*)(?<!-)-?{}'.format(suffix)):
            string_list = preserve(str(match[1])).split('_')
            field_list = parse_fields(string_list)
            total_dict[tuple(field_list)] = value

        for field_tuple, value in sorted(total_dict.items(), key=lambda k: k[0][-1]):
            field_list = list(field_tuple)
            index = field_list.pop()
            if index in entities:
                if index not in entity_index:
                    entity_index.append(index)
                entity_dict[tuple(field_list)].append(value)
            elif index in relations:
                if index not in relation_index:
                    relation_index.append(index)
                relation_dict[tuple(field_list)].append(value)
            elif not index:
                other_dict[tuple(field_list)] = value
            else:
                assert False, 'Not here, sir!'

            if index in er and index not in P:
                try:
                    match, count = next(pop_regex(stat_result, r'^.*(?i){}$'.format(er[index])))
                    P[index] = count
                except StopIteration as e:
                    print(index)
                    print(entities[index])
                    print(r'^.*(?i){}$'.format(entities[index]))
                    print(stat_result)
                    raise e

            #print(field_tuple)
            if field_tuple == ('', 'validation', '', index) and suffix.lower() == 'p':
                TP[index] = value * P[index]
            if field_tuple == ('', 'validation', '', index) and suffix.lower() == 'r':
                FP[index] = (1. / value - 1) * TP[index]
            if field_tuple == ('', 'validation', 'i', index) and suffix.lower() == 'p':
                TPi[index] = value * P[index]
            if field_tuple == ('', 'validation', 'i', index) and suffix.lower() == 'r':
                FPi[index] = (1. / value - 1) * TPi[index]

        entity_table = pd.DataFrame.from_dict(entity_dict)
        entity_table = entity_table.append(entity_table.mean(numeric_only=True), ignore_index=True)
        entity_table.index = entity_index + ['mean',]

        relation_table = pd.DataFrame.from_dict(relation_dict)
        relation_table = relation_table.append(relation_table.mean(numeric_only=True), ignore_index=True)
        relation_table.index = relation_index + ['mean',]

        other_tabel = pd.DataFrame.from_dict(other_dict, orient='index')
        if entity_dict: display(HTML(entity_table.to_html()))
        if relation_dict: display(HTML(relation_table.to_html()))
        if other_dict: display(HTML(other_tabel.to_html(header=False,)))

    #pprint(P)
    #pprint(TP)
    #pprint(FP)
    P_entities = sum([P[key] for key in entities])
    P_relations = sum([P[key] for key in relations])
    print('micro (no inference)')
    TP_entities = sum([TP[key] for key in entities])
    FP_entities = sum([FP[key] for key in entities])
    p = TP_entities / P_entities
    r = TP_entities / (TP_entities + FP_entities)
    print('F1 entities', 2 * p * r / (p + r))
    print('P entities', p)
    print('R entities', r)
    TP_relations = sum([TP[key] for key in relations])
    FP_relations = sum([FP[key] for key in relations])
    p = TP_relations / P_relations
    r = TP_relations / (TP_relations + FP_relations)
    print('F1 relations', 2 * p * r / (p + r))
    print('P relations', p)
    print('R relations', r)

    print('micro (inference)')
    TPi_entities = sum([TPi[key] for key in entities])
    FPi_entities = sum([FPi[key] for key in entities])
    pi = TPi_entities / P_entities
    ri = TPi_entities / (TPi_entities + FPi_entities)
    print('F1 entities', 2 * pi * ri / (pi + ri))
    print('P entities', pi)
    print('R entities', ri)
    TPi_relations = sum([TPi[key] for key in relations])
    FPi_relations = sum([FPi[key] for key in relations])
    pi = TPi_relations / P_relations
    ri = TPi_relations / (TPi_relations + FPi_relations)
    print('F1 relations', 2 * pi * ri / (pi + ri))
    print('P relations', pi)
    print('R relations', ri)

    display(Markdown('### What\'s left'))
    left_tabel = pd.DataFrame.from_dict(metrics, orient='index')
    display(HTML(left_tabel.to_html(header=False,)))
    
    return (P_entities, P_relations,
            TP_entities, TP_relations,
            FP_entities, FP_relations,
            TPi_entities, TPi_relations,
            FPi_entities, FPi_relations)


In [4]:
# log_path = './log.archive/log.20190801-024113/'
# show_setup(log_path)
# show_result(log_path)

In [8]:
master_path = './log.best.leakyrelu/'
P_entities_list = []
P_relations_list = []
TP_entities_list = []
TP_relations_list = []
FP_entities_list = []
FP_relations_list = []
TPi_entities_list = []
TPi_relations_list = []
FPi_entities_list = []
FPi_relations_list = []
for log_path in glob(os.path.join(master_path, 'log.*')):
    pprint(log_path)
    if os.path.isdir(log_path):
        config = show_setup(log_path)
        (P_entities, P_relations,
         TP_entities, TP_relations,
         FP_entities, FP_relations,
         TPi_entities, TPi_relations,
         FPi_entities, FPi_relations) = show_result(log_path, config)
        P_entities_list.append(P_entities)
        P_relations_list.append(P_relations)
        TP_entities_list.append(TP_entities)
        TP_relations_list.append(TP_relations)
        FP_entities_list.append(FP_entities)
        FP_relations_list.append(FP_relations)
        TPi_entities_list.append(TPi_entities)
        TPi_relations_list.append(TPi_relations)
        FPi_entities_list.append(FPi_entities)
        FPi_relations_list.append(FPi_relations)

def mm(P, TP, FP):
    p = sum(P)
    tp = sum(TP)
    fp = sum(FP)
    pr = tp / p
    rc = tp / (tp + fp)
    f1 = 2 * pr * rc / (pr + rc)
    return f1, pr, rc

print('Totle micro')
print('no inference')
f1, pr, rc = mm(P_entities_list, TP_entities_list, FP_entities_list)
print(f1, pr, rc)
f1, pr, rc = mm(P_relations_list, TP_relations_list, FP_relations_list)
print(f1, pr, rc)
print('inference')
f1, pr, rc = mm(P_entities_list, TPi_entities_list, FPi_entities_list)
print(f1, pr, rc)
f1, pr, rc = mm(P_relations_list, TPi_relations_list, FPi_relations_list)
print(f1, pr, rc)


'./log.best.leakyrelu/log.20190802-074627'


## Setup

{'Data': {'relative_path': 'data/EntityMentionRelation',
          'train_path': 'conll04.corp_3_train.corp',
          'valid_path': 'conll04.corp_3_test.corp'},
 'Model': {'activation': LeakyReLU(negative_slope=0.01),
           'compact': Namespace('layers':[48]),
           'distance_emb_size': 8,
           'dropout': 0.3,
           'embedding_dim': 8,
           'graph': {'balance_factor': 1.5,
                     'focal_gamma': 2,
                     'inference_interval': 100,
                     'inference_training_set': False,
                     'label_smoothing': 0.01},
           'max_distance': 64,
           'ngram': 5,
           'pretrained_dims': Namespace('word':50),
           'pretrained_files': Namespace('word':data/glove.6B/glove.6B.50d.txt),
           'relconv': Namespace('layers':[None, None, None],'kernel_size':7),
           'relemb': Namespace('emb_size':256),
           'rnn': Namespace('layers':2,'bidirectional':True)},
 'Train': {'iterator': Namespac

## Result

### Global information

...

"(, training, )",0.007011
"(, validation, )",0.097879
"(best, validation, )",0.061502


micro (no inference)
F1 entities 0.8137015113165328
P entities 0.8218287556464302
R entities 0.80573343711228
F1 relations 0.47306163130194284
P relations 0.4277829002414525
R relations 0.5290600545991905
micro (inference)
F1 entities 0.8160992698450831
P entities 0.8303892620807064
R entities 0.8022927839275104
F1 relations 0.4667356121159803
P relations 0.4408438103093917
R relations 0.49585856059521904


### What's left

training_\n[ epoch ],99
validation_\n[ epoch ],99
best_validation_\n[ epoch ],11


Totle micro
no inference
0.8128904021680999 0.8221348941246259 0.8038514977759641
0.46522431318921237 0.43050145560118835 0.5060398541367247
inference
0.8156614970080476 0.8300713685526366 0.8017433933987831
0.46161819005826055 0.4426581872830866 0.48227506914010126
